# EDS 220 Fall 2022
## Exploring Land Use Datasets

This notebook will give us a chance to load and play with data from the National Land Cover Database using Google Earth Engine. This exercise has been adapted from:

https://geemap.org/notebooks/nlcd_datapane/

We'll use "standard" Google Earth Engine packages to display this information:

In [1]:
# Import packages
import ee
import geemap

In [4]:
# Authenticate and initialize Google Earth Engine
#ee.Authenticate()
ee.Initialize()

The latest version of the NLCD available in Earth Engine is NLCD 2019:

https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD_RELEASES_2019_REL_NLCD

First, we can load the image collection holding that database:

In [38]:
# Load NLCD 2019 data collection
nlcd_dat = ee.ImageCollection("USGS/NLCD_RELEASES/2019_REL/NLCD")
nlcd_first = nlcd_dat.first()

nlcd_first.getInfo()

{'type': 'Image',
 'bands': [{'id': 'landcover',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [161190, 104424],
   'crs': 'PROJCS["IMAGINE GeoTIFF Support\nERDAS IMAGINE 2018  16.5.0.596\nProjection = Albers Conical Equal Area", \n  GEOGCS["WGS 84", \n    DATUM["WGS_1984", \n      SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], \n      AUTHORITY["EPSG","6326"]], \n    PRIMEM["Greenwich", 0.0], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH], \n    AUTHORITY["EPSG","4326"]], \n  PROJECTION["Albers_Conic_Equal_Area"], \n  PARAMETER["central_meridian", -96.0], \n  PARAMETER["latitude_of_origin", 23.0], \n  PARAMETER["standard_parallel_1", 29.5], \n  PARAMETER["false_easting", 0.0], \n  PARAMETER["false_northing", 0.0], \n  PARAMETER["standard_parallel_2", 45.5], \n  UNIT["m", 1.0], \n  AXIS["x", EAST], \n  AXIS["y", NORTH]]',
   'crs_transform':

The link above lets us explore the metadata associated with that image collection, which reveals that there are 8 different epochs included in this database: 2001, 2004, 2006, 2008, 2011, 2013, 2016, and 2019.

For each of the different epochs, the data has been stored as a single image - so we'll eventually want to load the epochal data as one image, rather than an ImageCollection, using the `.first()` method.

An efficient way to load in all of the data for each epoch is to define a _function_ which takes a year as its argument, and loads in the NLCD information corresponding to that year as a variable which can then be passed as a layer to GEE's mapping module. This is what the code block below does:

In [27]:
# Get an NLCD image by year.
def getNLCD(year):
    
    # Import the NLCD collection.
    dataset = ee.ImageCollection("USGS/NLCD_RELEASES/2019_REL/NLCD")

    # Filter the collection by year.
    nlcd = dataset.filter(ee.Filter.eq('system:index', year)).first()

    # Select the land cover band.
    landcover = nlcd.select('landcover')
    
    # Return a variable containing information from the land cover band
    return landcover



Another preparatory step is to create a base map on which we can plot the data we'll extract:

In [28]:
# Create and plot basemap
Map = geemap.Map(center=[34.42, -119.69], zoom = 8)

Map

Map(center=[34.42, -119.69], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

Now that our NLCD data extraction function has been defined, we can use it! First, we define an array containing all the years for which we would like to view data.

Then, we loop over each element in that year array, extract the NLCD information, and pass it to the GEE basemap.

In [29]:
# Define array of years corresponding to NLCD epochs

array_years = ['2001', '2004', '2006', '2008', '2011','2013','2016', '2019']

# Loop over years, call function to extract NLCD data

for i in array_years:
    data = getNLCD(i)
    Map.addLayer(data, {}, 'NLCD' + i)

This results in a very colorful map which can be hard to interpret without a legend. Fortunately, the NLCD color scheme is one of the legends that GEE has hard-coded as a built-in legend! The full list of legend options is available here:

https://geemap.org/notebooks/14_legends/

In [39]:
# Add the NLCD legend to the map.

Map.add_legend(legend_title = 'land cover class', builtin_legend = 'NLCD')